In the program, we have the last digit of sid `7`, so the order no. is `7`,$S1$ is 688 *China Overseas*, initial stock price `12.18`, volatility `43.6%`; $S2$ is 857 *Petrochina*, initial stock price `6.03`, volatility `30.0%`; $S1/S2$ correlation coefficient `0.304`, Group 3.

For Group 3, the $F\%=102.6\%, UB\%=130.0\%, A\%=122.0\%$

### Q2

The payoff of an *Average Worst-of Put Option* with two stocks *S1* and *S2* is based on the following formula:  $\max (100\% – A, 0)$ payable at maturity (*t = T = 0.75* year from start date). 

where: 

1. $S_{1,0}, S_{2,0}$ = stock price at time $t=0$
2. $S_{1,1}, S_{2,1}$ = stock price at time $t=0.25$ year
3. $S_{1,2}, S_{2,2}$ = stock price at time $t=0.75$ year
4. $A=(B_1+B_2)/2$
5. $B_1=\min (S_{1,1}/S_{1,0}, S_{2,1}/S_{2,0})$
6. $B_2=\min (S_{1,2}/S_{1,0}, S_{2,2}/S_{2,0})$

Continuously compounded interest rate *r = 4.17% p.a*. 

Calculate the fair price of the option as of the start date (time *t=0*). 

Note: The answers should be a percentage (or a decimal number) smaller than 30%, and there is no need to multiply the answers with *S1* and/or *S2*. 

In [1]:
import numpy as np

S1_0 = 12.18
S2_0 = 6.03
sigma1 = 43.6 / 100
sigma2 = 30.0 / 100
rho = 0.304
r = 4.17 / 100
T = 0.75
t1 = 0.25
t2 = T - t1

num_simulations = 1_000_000  # 模拟次数

corr_matrix = np.array([[1, rho], [rho, 1]])
# L = np.linalg.cholesky(corr_matrix)

# generate epsilons
epsilon1 = np.random.normal(0, 1, (num_simulations, 2))
epsilon1 = epsilon1 @ corr_matrix.T # L.T
S1_1 = S1_0 * np.exp((r - 0.5 * sigma1**2) * t1 + sigma1 * np.sqrt(t1) * epsilon1[:, 0])
S2_1 = S2_0 * np.exp((r - 0.5 * sigma2**2) * t1 + sigma2 * np.sqrt(t1) * epsilon1[:, 1])

epsilon2 = np.random.normal(0, 1, (num_simulations, 2))
epsilon2 = epsilon2 @ corr_matrix.T # L.T
S1_2 = S1_1 * np.exp((r - 0.5 * sigma1**2) * t2 + sigma1 * np.sqrt(t2) * epsilon2[:, 0])
S2_2 = S2_1 * np.exp((r - 0.5 * sigma2**2) * t2 + sigma2 * np.sqrt(t2) * epsilon2[:, 1])

# calculate A, B and payoff
B1 = np.minimum(S1_1 / S1_0, S2_1 / S2_0)
B2 = np.minimum(S1_2 / S1_0, S2_2 / S2_0)
A = (B1 + B2) / 2
payoff = np.maximum(1 - A, 0)

# calculate the current value
discount_factor = 1 / (1 + r * T) # np.exp(-r * T)
option_price = discount_factor * np.mean(payoff)

print(f"Fair price of the option: {option_price * 100:.2f}%")

Fair price of the option: 12.53%


### Q2(i)

Use a Monte Carlo scheme with time steps *N = 150*, i.e. $\Delta t=T/N=1/200$ (refer to the discretization scheme in Topic 1-2, slides 37 and 38).  Give the answers with: (a) 10000 paths; (b) 300000 paths.  Record the computation times in each case. 

[Note: in this part, don’t use the exact discretization scheme.  Marks will be deducted if the exact scheme is adopted.] 

In [ ]:
import numpy as np
import time

# Given parameters
S1_0 = 12.18  # Initial stock price for S1
S2_0 = 6.03   # Initial stock price for S2
r = 4.17 / 100    # Continuously compounded interest rate
sigma1 = 43.6 / 100  # Volatility for S1
sigma2 = 30.0 / 100  # Volatility for S2
S1S2coef = 0.304   # Correlation coefficient between S1 and S2
T = 0.75      # Maturity in years
N = 150       # Number of time steps
dt = 1/200    # Time step size

# Number of paths for the simulation
num_paths_1 = 10_000
num_paths_2 = 300_000

# Function to generate correlated Brownian motions
def generate_correlated_brownian_motion(num_paths, S1S2coef, sigma1, sigma2, dt):
    z1 = np.random.normal(0, 1, (num_paths, N))
    z2 = S1S2coef * z1 + np.sqrt(1 - S1S2coef**2) * np.random.normal(0, 1, (num_paths, N))
    return z1 * np.sqrt(dt), z2 * np.sqrt(dt)

# Function to perform Monte Carlo simulation
def monte_carlo_option_price(num_paths):
    start_time = time.time()
    
    # Generate correlated Brownian motions
    z1, z2 = generate_correlated_brownian_motion(num_paths, S1S2coef, sigma1, sigma2, dt)
    
    # Simulate stock prices
    S1 = np.zeros((num_paths, N + 1))
    S2 = np.zeros((num_paths, N + 1))
    S1[:, 0] = S1_0
    S2[:, 0] = S2_0
    
    for i in range(1, N + 1):
        S1[:, i] = S1[:, i - 1] * np.exp((r - 0.5 * sigma1**2) * dt + sigma1 * z1[:, i - 1])
        S2[:, i] = S2[:, i - 1] * np.exp((r - 0.5 * sigma2**2) * dt + sigma2 * z2[:, i - 1])
    
    # Calculate B1 and B2
    B1 = np.minimum(S1[:, 75] / S1_0, S2[:, 75] / S2_0)
    B2 = np.minimum(S1[:, -1] / S1_0, S2[:, -1] / S2_0)
    
    # Calculate A
    A = (B1 + B2) / 2
    
    # Calculate payoff
    payoff = np.maximum(1 - A, 0)
    
    # Discount the average payoff
    option_price = np.exp(-r * T) * np.mean(payoff)
    
    computation_time = time.time() - start_time
    return option_price, computation_time

# Run the simulation for 10,000 and 300,000 paths
option_price_1, computation_time_1 = monte_carlo_option_price(num_paths_1)
option_price_2, computation_time_2 = monte_carlo_option_price(num_paths_2)

print(f"Option price with {num_paths_1} paths: {option_price_1 * 100}%")
print(f"Computation time for {num_paths_1} paths: {computation_time_1} seconds")
print(f"Option price with {num_paths_2} paths: {option_price_2 * 100}%")
print(f"Computation time for {num_paths_2} paths: {computation_time_2} seconds")

Option price with 10000 paths: 14.353848259555244%
Computation time for 10000 paths: 0.10100412368774414 seconds
Option price with 300000 paths: 14.404362040686175%
Computation time for 300000 paths: 4.921932935714722 seconds


### Q2(ii)

Use a Monte Carlo scheme with two time steps N = 2, $\Delta t_1 = 0.25, \Delta t_2 = 0.5$ (refer to the discretization scheme in Topic 1-2, slides 39, 40, 42).  Give the answers with: (a) 10000 paths; (b) 300000 paths.  Record the computation times in each case. 

In [2]:
import numpy as np
import time

# Given parameters
S10 = 12.18  # Initial stock price for S1
S20 = 6.03   # Initial stock price for S2
r = 0.0417   # Continuously compounded interest rate
sigma1 = 0.436  # Volatility for S1
sigma2 = 0.30   # Volatility for S2
S1S2coef = 0.304     # Correlation coefficient between S1 and S2
T = 0.75       # Maturity in years
N = 2          # Number of time steps
dt1 = 0.25     # Time step 1
dt2 = 0.5      # Time step 2
numpaths1 = 10000  # Number of paths for case (a)
numpaths2 = 300000 # Number of paths for case (b)

# Simulation function
def montecarlooptionprice(numpaths):
    start_time = time.time()
    
    # Preallocate arrays for performance
    S1_paths = np.zeros((numpaths, N+1))
    S2_paths = np.zeros((numpaths, N+1))
    S1_paths[:, 0] = S10
    S2_paths[:, 0] = S20
    
    # Generate paths
    for i in range(numpaths):
        Z1 = np.random.normal(0, 1)
        Z2 = S1S2coef * Z1 + np.sqrt(1 - S1S2coef**2) * np.random.normal(0, 1)
        S1_paths[i, 1] = S1_paths[i, 0] * np.exp((r - 0.5 * sigma1**2) * dt1 + sigma1 * np.sqrt(dt1) * Z1)
        S2_paths[i, 1] = S2_paths[i, 0] * np.exp((r - 0.5 * sigma2**2) * dt1 + sigma2 * np.sqrt(dt1) * Z2)
        
        Z1 = np.random.normal(0, 1)
        Z2 = S1S2coef * Z1 + np.sqrt(1 - S1S2coef**2) * np.random.normal(0, 1)
        S1_paths[i, 2] = S1_paths[i, 1] * np.exp((r - 0.5 * sigma1**2) * dt2 + sigma1 * np.sqrt(dt2) * Z1)
        S2_paths[i, 2] = S2_paths[i, 1] * np.exp((r - 0.5 * sigma2**2) * dt2 + sigma2 * np.sqrt(dt2) * Z2)
    
    # Calculate B1 and B2
    B1 = np.minimum(S1_paths[:, 1] / S10, S2_paths[:, 1] / S20)
    B2 = np.minimum(S1_paths[:, 2] / S10, S2_paths[:, 2] / S20)
    
    # Calculate A
    A = (B1 + B2) / 2
    
    # Calculate payoff
    payoff = np.maximum(1 - A, 0)
    
    # Discount the average payoff
    option_price = np.exp(-r * T) * np.mean(payoff)
    
    computation_time = time.time() - start_time
    return option_price, computation_time

# Run the simulation for both cases
option_price1, computation_time1 = montecarlooptionprice(numpaths1)
option_price2, computation_time2 = montecarlooptionprice(numpaths2)

# Print the results
print(f"Option price with {numpaths1} paths: {option_price1 * 100}%")
print(f"Computation time for {numpaths1} paths: {computation_time1} seconds")
print(f"Option price with {numpaths2} paths: {option_price2 * 100}%")
print(f"Computation time for {numpaths2} paths: {computation_time2} seconds")

Option price with 10000 paths: 13.301575248746508%
Computation time for 10000 paths: 0.10353612899780273 seconds
Option price with 300000 paths: 13.139970021203442%
Computation time for 300000 paths: 2.9412877559661865 seconds
